In [1]:
!wget -nc https://www.dropbox.com/s/gmgus2rm4sks6b8/whitenoise.h5

--2024-10-22 16:53:01--  https://www.dropbox.com/s/gmgus2rm4sks6b8/whitenoise.h5
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:18::a27d:4612, 162.125.70.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:18::a27d:4612|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/vfa73xjnggeo2tnuhbyka/whitenoise.h5?rlkey=awkqy3a8lj86vhvy3gb9snzkw [following]
--2024-10-22 16:53:02--  https://www.dropbox.com/scl/fi/vfa73xjnggeo2tnuhbyka/whitenoise.h5?rlkey=awkqy3a8lj86vhvy3gb9snzkw
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucba24529c7839137ce06a70e299.dl.dropboxusercontent.com/cd/0/inline/Cc66AS1_OXpPXkAiTW4nTrPqLfVCQ_rk88lCrO9KrorF_MiXDEaCLofsbLPViGcKhNjnlDe-RbFRakrkLv1WzU8iJo10YbDsVSJ6gbVtx9V54CqpaMa-NobbXCuyGIuKSQk/file# [following]
--2024-10-22 16:53:03--  https://ucba24529c7839137ce06a70e299.dl.dropboxusercontent.com/cd/0/inline/

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.distributions import Poisson


import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import trange
from copy import deepcopy


# Specify that we want our tensors on the GPU and in float32
device = torch.device('cuda')
dtype = torch.float32
# Load the white noise data
f = h5py.File("whitenoise.h5", mode='r')
frame_rate = 100
times = torch.tensor(f['train']['time'][:], dtype=dtype)
stimulus = torch.tensor(f['train']['stimulus'][:], dtype=torch.uint8)
spikes = torch.tensor(f['train']['response']['binned'][:].T, dtype=dtype)
test_times = torch.tensor(f['test']['time'][:], dtype=dtype)
test_stimulus = torch.tensor(f['test']['stimulus'][:], dtype=torch.uint8)
test_rates = torch.tensor(f['test']['response']['binned'][:, :-1].T, dtype=dtype)

# Get the size of the training data
num_frames, height, width = stimulus.shape
_, num_neurons = spikes.shape


/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
num_frames
num_neurons

9